In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!unzip -q "/content/drive/MyDrive/Colab Notebooks/data-emotion-recognition/raw_data/fer13.zip"
# -d "/content/drive/MyDrive/Colab Notebooks/data-emotion-recognition/raw_data"

In [ ]:
import sys
sys.path.insert(0,"/content/test")
sys.path.insert(0,"/content/train")
sys.path.insert(0,"/content/drive/MyDrive/Colab Notebooks/data-emotion-recognition/")

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import models, Sequential, layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf
import decode.ml_logic.emotion_detection.emotion_baseline as ed

2023-05-27 11:04:30.458392: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
DIR_TRAIN = "/content/train"
DIR_TEST = "/content/test"

train_ds, val_ds = ds = image_dataset_from_directory(
        DIR_TRAIN,
        shuffle=True,
        seed=42,
        labels="inferred",
        label_mode = "categorical",
        validation_split = 0.3, 
        subset = "both",
        image_size=(95,95),
        color_mode="rgb",
        batch_size=32,
    )

test_ds = image_dataset_from_directory(
        DIR_TEST,
        shuffle=True,
        seed=42,
        labels="inferred",
        label_mode = "categorical",
        image_size=(95,95),
        color_mode="rgb",
        batch_size=32,
    )

def preprocess(images, labels):
  return preprocess_input(images), labels

train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)
test_ds = test_ds.map(preprocess)
#next(iter(train_ds))

In [ ]:
def emotion_recognition(input_shape):
    
    model = Sequential()    
    model.add(Input(shape=input_shape)) # Input tensor
    
    model.add(layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='valid')) 
    model.add(layers.BatchNormalization(axis=3)) 
    model.add(layers.Activation(activation='relu')) #output layer 
              
    model.add(layers.Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same')) 
    model.add(layers.BatchNormalization(axis=3)) 
    model.add(layers.Activation(activation='relu')) #output layer 

    model.add(layers.MaxPooling2D((2,2)))          
              
    model.add(layers.Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='valid')) 
    model.add(layers.BatchNormalization(axis=3)) 
    model.add(layers.Activation(activation='relu')) #output layer 
              
    model.add(layers.Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same')) 
    model.add(layers.BatchNormalization(axis=3)) 
    model.add(layers.Activation(activation='relu')) #output layer 
              
    model.add(layers.MaxPooling2D((2,2)))                
              
    model.add(layers.Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='valid')) 
    model.add(layers.BatchNormalization(axis=3)) 
    model.add(layers.Activation(activation='relu')) #output layer 
              
    model.add(layers.MaxPooling2D((2,2)))  
              
    model.add(layers.Flatten())                  
    model.add(layers.Dense(200, activation='relu'))      
    model.add(layers.Dropout())                             
    model.add(layers.Dense(7, activation='softmax'))                    
    return model

In [ ]:


base_model = InceptionResNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(95, 95, 3)
)
base_model.trainable = False

model = Sequential(
    [
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.6),
        layers.Dense(7, activation="softmax"),
    ]
)

model.summary()

es = EarlyStopping(patience=5, restore_best_weights=True)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)

adam = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

#use this for ohe-labels
model.compile(optimizer=adam, loss="CategoricalCrossentropy", metrics=["Accuracy", "CategoricalAccuracy"])
#model.compile(optimizer=adam, loss="SparseCategoricalCrossentropy", metrics=["SparseCategoricalAccuracy"])

#X_train, y_train = ed.get_first_batches(train_ds, 898);
#X_test, y_test = ed.get_first_batches(test_ds, 225);
#y_cat_train = to_categorical(y_train, num_classes=7)
#y_cat_test = to_categorical(y_test, num_classes=7)


#history = model.fit(X_train_preproc, y_cat_train, epochs=5, batch_size=32, callbacks=[es], validation_split=0.3)
history = model.fit(train_ds, epochs=50, batch_size=32, callbacks=[es], validation_data=val_ds)